In [7]:
from typing import List
import numpy as np


def main():
  before_items = ["apple", "banana", "grape", "orange", "peach"]
  after_items = ["apple", "banana", "peach", "orange", "grape"]

  k = 5  # 評価するランキングの長さ
  y_true, y_score = create_relevance(before_items, after_items, k)
  result = ndcg(y_true, y_score, k)

  print(f"k = {k}")
  print(f"y_true  = {y_true}")
  print(f"y_score = {y_score}")
  print(f"ndcg = {result}")

def create_relevance(before_ids: List[str], after_ids: List[str], k: int) -> tuple[np.ndarray, np.ndarray]:
  """
  変更前のランキングと変更後のランキングから擬似的に関連度を生成する
  """
  # 各要素にランキング降順でスコアを付与する
  scores = {id: (k - i) for i, id in enumerate(before_ids[:k])}
  # 変更前のランキングの関連度を擬似的に生成 = [k, k-1, ..., 1]
  y_true = np.array([list(scores.values())])
  # 変更後のランキングの関連度を擬似的に生成
  y_score = np.array([[scores.get(id, 0) for id in after_ids[:k]]])

  return y_true, y_score


def ndcg(
  y_true: np.ndarray, y_score: np.ndarray, k: int) -> float:
  """
  Normalized DCG (nDCG)
  """
  y_true = np.asfarray(y_true)
  y_score = np.asfarray(y_score)
  # DCG
  dcg_val = _dcg(y_score, k=k)
  # IDCG
  idcg_val = _dcg(y_true, k=k)

  return dcg_val / idcg_val if idcg_val > 0 else 0.0


def _dcg(
  rels: np.ndarray, k: int) -> float:
  """
  Discounted Cumulative Gain (DCG)
  """
  # 関連度
  rels = np.asfarray(rels[:k])
  # 各要素の割引値を計算
  discounts = np.log2(np.arange(2, rels.size + 2))
  return np.sum(rels / discounts)


if __name__ == "__main__":
  main()

k = 5
y_true  = [[5 4 3 2 1]]
y_score = [[5 4 1 2 3]]
ndcg = 0.9779696223537329


In [12]:
from typing import List
import numpy as np

# RBPに必要なライブラリをインポート
# import numpy as np # すでにインポートされています
# from typing import List # すでにインポートされています

def create_relevance(before_ids: List[str], after_ids: List[str], k: int) -> tuple[np.ndarray, np.ndarray]:
    """
    変更前のランキングと変更後のランキングから擬似的に関連度を生成する
    （元のコードと同じロジックを使用）
    """
    # 各要素にランキング降順でスコアを付与する
    # before_items = ["apple", "banana", "grape", "orange", "peach"]
    # after_items = ["kiwi", "mango", "pineapple", "strawberry", "watermelon"]
    # k=4の場合,scores = {"apple": 4, "banana": 3, "grape": 2, "orange": 1}
    scores = {id: (k - i) for i, id in enumerate(before_ids[:k])}
    
    # 変更前のランキングの関連度を擬似的に生成 = [k, k-1, ..., 1]
    y_true = np.array([list(scores.values())])
    
    # 変更後のランキングの関連度を擬似的に生成
    # after_itemsがscoresに存在しないため,scores.get(id, 0)により全て 0 になる
    y_score = np.array([[scores.get(id, 0) for id in after_ids[:k]]])

    return y_true, y_score


def rbp(
    rels: np.ndarray, p: float, k: int) -> float:
    """
    Rank-Biased Precision (RBP)
    """
    # 関連度
    rels_k = np.asfarray(rels.flatten()[:k])
    
    # RBPの重み (p^(i-1)) を計算
    # 順位 i のインデックスは i-1 なので,0 から k-1 の指数を使用
    # weights = [p^0, p^1, p^2, ..., p^(k-1)]
    ranks = np.arange(k)
    weights = np.power(p, ranks)
    
    # RBPの式: (1 - p) * sum(rel_i * p^(i-1))
    rbp_score = (1 - p) * np.sum(rels_k * weights)
    
    return float(rbp_score)

def main():
    before_items = ["apple", "banana", "grape", "orange", "peach"]
    after_items = ["apple", "banana", "orange", "grape", "peach"]


    
    k = 5  # 評価するランキングの長さ
    p = 0.95 # RBPの持続性パラメータ
    
    # 擬似関連度の生成
    # y_true = [4, 3, 2, 1] (理想ランキングの関連度)
    # y_score = [0, 0, 0, 0] (変更後のランキングの関連度 - 関連文書がないため)
    y_true, y_score = create_relevance(before_items, after_items, k)
    
    # --- RBPの計算 ---
    # RBPは正規化を式に含むため,IDCGに相当する計算は不要
    
    # 1. 変更後のランキングのRBP (y_score)
    result_rbp_score = rbp(y_score, p, k)
    
    # 2. 理想的なランキングのRBP (y_true) - 正規化係数 (IRBP)
    # RBPは (1-p) で正規化されていますが,ここではy_trueのRBP値を算出
    result_rbp_true = rbp(y_true, p, k)

    # 3. RBPを理想RBP (IRBP) で割って正規化 (Optional: nDCGとの比較のため)
    # ただし,RBPの定義上,(1-p)の係数により最大値は1-p*p^kの近似値となる
    # ここでは,純粋なRBPの値を結果として出力
    
    print(f"--- RBP ({p=}, {k=}) ---")
    print(f"y_true (理想の関連度) = {y_true.flatten()[:k]}")
    print(f"y_score (実際の関連度) = {y_score.flatten()[:k]}")
    print(f"RBP (理想ランキング) = {result_rbp_true:.4f}")
    print(f"RBP (評価ランキング) = {result_rbp_score:.4f}")

# main() # 実行すると結果が表示されます

if __name__ == "__main__":
    main()

--- RBP (p=0.95, k=5) ---
y_true (理想の関連度) = [5 4 3 2 1]
y_score (実際の関連度) = [5 4 2 3 1]
RBP (理想ランキング) = 0.7018
RBP (評価ランキング) = 0.6996
